In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import tkinter as tk
from tkinter import *
from PIL import Image, ImageDraw, ImageOps
import joblib

In [2]:
mnist = fetch_openml('mnist_784')

C:\anaconda\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
x,y = mnist.data, mnist.target.astype('int')

In [4]:
x.shape

(70000, 784)

In [5]:
y.shape

(70000,)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [7]:
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_test shape: {y_test.shape}")

x_train shape: (56000, 784)
y_train shape: (56000,)
x_test shape: (14000, 784)
y_test shape: (14000,)


In [8]:
gb_model =GradientBoostingClassifier(n_estimators = 100, random_state = 42)

In [ ]:
gb_model.fit(x_train, y_train)
gb_predictions = gb_model.predict(x_test)

gb_accuracy = accuracy_score(y_test, gb_predictions)

gb_accuracy

In [ ]:
joblib.dump(gb_model, 'rf_mnist_model.pkl')

In [ ]:
# Evaluate the model (optional)
print(f"Model accuracy: {gb_model.score(x_test, y_test):.2f}")

In [ ]:
rf_model = joblib.load('gb_mnist_model.pkl')

In [ ]:
class DrawDigitsApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Draw a Digit")

        self.canvas = tk.Canvas(root, width=200, height=200, bg='white')
        self.canvas.pack()

        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)

        self.canvas.bind("<B1-Motion>", self.paint)
        self.clear_button = tk.Button(root, text="Clear", command=self.clear_canvas)
        self.clear_button.pack()
        self.submit_button = tk.Button(root, text="Submit", command=self.submit_canvas)
        self.submit_button.pack()
        self.result_label = tk.Label(root, text="")
        self.result_label.pack()

    def paint(self, event):
        x1, y1 = (event.x - 4), (event.y - 4)
        x2, y2 = (event.x + 4), (event.y + 4)
        self.canvas.create_oval(x1, y1, x2, y2, fill='black')
        self.draw.ellipse([x1, y1, x2, y2], fill='black')

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (200, 200), 255)
        self.draw = ImageDraw.Draw(self.image)

    def submit_canvas(self):
        # Invert the image colors to match MNIST format (black background and white digits)
        image_inverted = ImageOps.invert(self.image)

     # Resize the image to 28x28 as required by MNIST
        image_resized = image_inverted.resize((28, 28))

        # Normalize the image
        image_array = np.array(image_resized) / 255.0

        # Flatten the array
        image_array = image_array.reshape(1, -1)

        # Predict the digit using the pre-trained RandomForest model
        prediction = rf_model.predict(image_array)
        digit = prediction[0]

        # Display the result
        self.result_label.config(text=f"Predicted Digit: {digit}")

In [ ]:
if __name__ == "__main__":
    root = tk.Tk()
    app = DrawDigitsApp(root)
    root.mainloop()